In [ ]:
## all_stunts.csv
all_stunts = pd.read_csv('/kaggle/input/all-stunts/finalplays_with_epa.csv')
## big data bowl games.csv
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/games.csv')
games = games[['gameId','homeTeamAbbr','visitorTeamAbbr']]
all_stunts = all_stunts.merge(games,on='gameId',how = 'left')
all_stunts['yardsToEZ'] = all_stunts.absoluteYardlineNumber - 10
all_stunts['pointDifferential'] = np.where(all_stunts.possessionTeam == all_stunts.homeTeamAbbr,
                                   all_stunts.preSnapVisitorScore - all_stunts.preSnapHomeScore,
                                   all_stunts.preSnapHomeScore - all_stunts.preSnapVisitorScore)
all_stunts['min_sec'] = all_stunts.gameClock.map(lambda x:x.split(':'))
all_stunts['quarterTimeLeft'] = all_stunts.min_sec.map(lambda x:(int(x[0])*60)+int(x[1]))
all_stunts['gameTimeLeft'] = (4 - all_stunts.quarter)*15*60 + all_stunts.quarterTimeLeft
all_stunts = all_stunts.rename(columns = {"('blitz', 'mean')":"blitz",
                             "('blitz_class', '')":"blitz_class",
                             "('stunt', '')":"stunt",
                             "('stunt_class', '')":"stunt_class"})
# all_stunts = all_stunts.drop(all_stunts.index[0])
all_stunts['gameId'] = all_stunts.gameId.astype('int64')
all_stunts['playId'] = all_stunts.playId.astype('int64')
all_stunts = all_stunts.drop(['Unnamed: 0'], axis = 1)
all_stunts = all_stunts.dropna(subset = ['yardsToEZ'])
all_stunts['gameplayId'] = all_stunts.apply(lambda row:str(row.loc['gameId'])+"_"+str(row.loc['playId']), axis=1)
all_stunts = all_stunts.drop(['...1','possessionTeam','yardlineNumber','yardlineSide','playDescription','preSnapHomeScore',
                              'preSnapVisitorScore','offenseFormation', 'personnelO', 'defendersInBox','personnelD', 'dropBackType',
                              'pff_passCoverage','pff_passCoverageType','gameId','playId','penaltyYards','prePenaltyPlayResult','foulName1', 'foulNFLId1',
                              'foulName2','foulNFLId2', 'foulName3','foulNFLId3','absoluteYardlineNumber','pff_playAction','visitorTeamAbbr','homeTeamAbbr',
                              'min_sec'],axis = 1)
all_stunts.iloc[0]

In [ ]:
def calculate_epa_blitzes(team, all_stunts):
    tm = all_stunts.loc[all_stunts.defensiveTeam == team]
    
    ## when stunting
    q = tm.loc[tm.blitz==1].groupby('quarter').epa.mean().values
    q1,q2,q3,q4 = q[0],q[1],q[2],q[3]

    first10 = tm.loc[(tm.down == 1)&(tm.yardsToGo==10)&(tm.blitz == 1)].epa.mean()
    secondShort = tm.loc[(tm.down == 2)&(tm.yardsToGo<5)&(tm.blitz == 1)].epa.mean()
    secondLong = tm.loc[(tm.down == 2)&(tm.yardsToGo>=5)&(tm.blitz == 1)].epa.mean()
    thirdShort = tm.loc[(tm.down == 3)&(tm.yardsToGo<5)&(tm.blitz == 1)].epa.mean()
    thirdLong = tm.loc[(tm.down == 3)&(tm.yardsToGo>=5)&(tm.blitz == 1)].epa.mean()
    fourthShort = tm.loc[(tm.down == 4)&(tm.yardsToGo<5)&(tm.blitz == 1)].epa.mean()
    fourthLong = tm.loc[(tm.down == 4)&(tm.yardsToGo>=5)&(tm.blitz == 1)].epa.mean()

    g2g = tm.loc[(tm.yardsToEZ < 10)&(tm.blitz==1)].epa.mean()
    redzone = tm.loc[(tm.yardsToEZ < 20)&(tm.blitz==1)].epa.mean()
    fgRange = tm.loc[(tm.yardsToEZ < 45)&(tm.blitz==1)].epa.mean()

    singleScoreW = tm.loc[(tm.pointDifferential <= 8)&(tm.pointDifferential > 0)&(tm.blitz == 1)].epa.mean()
    singleScoreL = tm.loc[(tm.pointDifferential >= -8)&(tm.pointDifferential < 0)&(tm.blitz == 1)].epa.mean()
    scoreT = tm.loc[(tm.pointDifferential == 0)&(tm.blitz == 1)].epa.mean()
    multScoreW = tm.loc[(tm.pointDifferential >= 8)&(tm.blitz == 1)].epa.mean()
    multScoreL = tm.loc[(tm.pointDifferential < -8)&(tm.blitz == 1)].epa.mean()

    epa_blitzing = np.array([first10,secondShort,secondLong,thirdShort,thirdLong,fourthShort,fourthLong,g2g,
                             redzone,fgRange,singleScoreW,singleScoreL,scoreT,multScoreW,multScoreL,q1,q2,q3,q4])

    ## when not blitzing
    q = tm.loc[tm.blitz==0].groupby('quarter').epa.mean().values
    q1,q2,q3,q4 = q[0],q[1],q[2],q[3]

    first10 = tm.loc[(tm.down == 1)&(tm.yardsToGo==10)&(tm.blitz == 0)].epa.mean()
    secondShort = tm.loc[(tm.down == 2)&(tm.yardsToGo<5)&(tm.blitz == 0)].epa.mean()
    secondLong = tm.loc[(tm.down == 2)&(tm.yardsToGo>=5)&(tm.blitz == 0)].epa.mean()
    thirdShort = tm.loc[(tm.down == 3)&(tm.yardsToGo<5)&(tm.blitz == 0)].epa.mean()
    thirdLong = tm.loc[(tm.down == 3)&(tm.yardsToGo>=5)&(tm.blitz == 0)].epa.mean()
    fourthShort = tm.loc[(tm.down == 4)&(tm.yardsToGo<5)&(tm.blitz == 0)].epa.mean()
    fourthLong = tm.loc[(tm.down == 4)&(tm.yardsToGo>=5)&(tm.blitz == 0)].epa.mean()

    g2g = tm.loc[(tm.yardsToEZ < 10)&(tm.blitz==0)].epa.mean()
    redzone = tm.loc[(tm.yardsToEZ < 20)&(tm.blitz==0)].epa.mean()
    fgRange = tm.loc[(tm.yardsToEZ < 45)&(tm.blitz==0)].epa.mean()

    singleScoreW = tm.loc[(tm.pointDifferential <= 8)&(tm.pointDifferential > 0)&(tm.blitz == 0)].epa.mean()
    singleScoreL = tm.loc[(tm.pointDifferential >= -8)&(tm.pointDifferential < 0)&(tm.blitz == 0)].epa.mean()
    scoreT = tm.loc[(tm.pointDifferential == 0)&(tm.blitz == 0)].epa.mean()
    multScoreW = tm.loc[(tm.pointDifferential >= 8)&(tm.blitz == 0)].epa.mean()
    multScoreL = tm.loc[(tm.pointDifferential < -8)&(tm.blitz == 0)].epa.mean()


    epa_not_blitzing = np.array([first10,secondShort,secondLong,thirdShort,thirdLong,fourthShort,fourthLong,g2g,
                             redzone,fgRange,singleScoreW,singleScoreL,scoreT,multScoreW,multScoreL,q1,q2,q3,q4])

    ## how much does the offense lose in epa when the defense blitzes vs. normal
    delta_epa = epa_blitzing - epa_not_blitzing

    return epa_blitzing,epa_not_blitzing,delta_epa

In [ ]:
def calculate_epa_stunts(team, all_stunts):
    tm = all_stunts.loc[all_stunts.defensiveTeam == team]
    
    ## when stunting
    q = tm.loc[tm.stunt==1].groupby('quarter').epa.mean().values
    q1,q2,q3,q4 = q[0],q[1],q[2],q[3]

    first10 = tm.loc[(tm.down == 1)&(tm.yardsToGo==10)&(tm.stunt == 1)].epa.mean()
    secondShort = tm.loc[(tm.down == 2)&(tm.yardsToGo<5)&(tm.stunt == 1)].epa.mean()
    secondLong = tm.loc[(tm.down == 2)&(tm.yardsToGo>=5)&(tm.stunt == 1)].epa.mean()
    thirdShort = tm.loc[(tm.down == 3)&(tm.yardsToGo<5)&(tm.stunt == 1)].epa.mean()
    thirdLong = tm.loc[(tm.down == 3)&(tm.yardsToGo>=5)&(tm.stunt == 1)].epa.mean()
    fourthShort = tm.loc[(tm.down == 4)&(tm.yardsToGo<5)&(tm.stunt == 1)].epa.mean()
    fourthLong = tm.loc[(tm.down == 4)&(tm.yardsToGo>=5)&(tm.stunt == 1)].epa.mean()

    g2g = tm.loc[(tm.yardsToEZ < 10)&(tm.stunt==1)].epa.mean()
    redzone = tm.loc[(tm.yardsToEZ < 20)&(tm.stunt==1)].epa.mean()
    fgRange = tm.loc[(tm.yardsToEZ < 45)&(tm.stunt==1)].epa.mean()

    singleScoreW = tm.loc[(tm.pointDifferential <= 8)&(tm.pointDifferential > 0)&(tm.stunt == 1)].epa.mean()
    singleScoreL = tm.loc[(tm.pointDifferential >= -8)&(tm.pointDifferential < 0)&(tm.stunt == 1)].epa.mean()
    scoreT = tm.loc[(tm.pointDifferential == 0)&(tm.stunt == 1)].epa.mean()
    multScoreW = tm.loc[(tm.pointDifferential >= 8)&(tm.stunt == 1)].epa.mean()
    multScoreL = tm.loc[(tm.pointDifferential < -8)&(tm.stunt == 1)].epa.mean()

    epa_stunting = np.array([first10,secondShort,secondLong,thirdShort,thirdLong,fourthShort,fourthLong,g2g,
                             redzone,fgRange,singleScoreW,singleScoreL,scoreT,multScoreW,multScoreL,q1,q2,q3,q4])

    ## when not stunting
    q = tm.loc[tm.stunt==0].groupby('quarter').epa.mean().values
    q1,q2,q3,q4 = q[0],q[1],q[2],q[3]

    first10 = tm.loc[(tm.down == 1)&(tm.yardsToGo==10)&(tm.stunt == 0)].epa.mean()
    secondShort = tm.loc[(tm.down == 2)&(tm.yardsToGo<5)&(tm.stunt == 0)].epa.mean()
    secondLong = tm.loc[(tm.down == 2)&(tm.yardsToGo>=5)&(tm.stunt == 0)].epa.mean()
    thirdShort = tm.loc[(tm.down == 3)&(tm.yardsToGo<5)&(tm.stunt == 0)].epa.mean()
    thirdLong = tm.loc[(tm.down == 3)&(tm.yardsToGo>=5)&(tm.stunt == 0)].epa.mean()
    fourthShort = tm.loc[(tm.down == 4)&(tm.yardsToGo<5)&(tm.stunt == 0)].epa.mean()
    fourthLong = tm.loc[(tm.down == 4)&(tm.yardsToGo>=5)&(tm.stunt == 0)].epa.mean()

    g2g = tm.loc[(tm.yardsToEZ < 10)&(tm.stunt==0)].epa.mean()
    redzone = tm.loc[(tm.yardsToEZ < 20)&(tm.stunt==0)].epa.mean()
    fgRange = tm.loc[(tm.yardsToEZ < 45)&(tm.stunt==0)].epa.mean()

    singleScoreW = tm.loc[(tm.pointDifferential <= 8)&(tm.pointDifferential > 0)&(tm.stunt == 0)].epa.mean()
    singleScoreL = tm.loc[(tm.pointDifferential >= -8)&(tm.pointDifferential < 0)&(tm.stunt == 0)].epa.mean()
    scoreT = tm.loc[(tm.pointDifferential == 0)&(tm.stunt == 0)].epa.mean()
    multScoreW = tm.loc[(tm.pointDifferential >= 8)&(tm.stunt == 0)].epa.mean()
    multScoreL = tm.loc[(tm.pointDifferential < -8)&(tm.stunt == 0)].epa.mean()


    epa_not_stunting = np.array([first10,secondShort,secondLong,thirdShort,thirdLong,fourthShort,fourthLong,g2g,
                             redzone,fgRange,singleScoreW,singleScoreL,scoreT,multScoreW,multScoreL,q1,q2,q3,q4])

    ## how much does the offense lose in epa when the defense stunts vs. normal
    delta_epa = epa_stunting - epa_not_stunting
    
    return epa_stunting,epa_not_stunting,delta_epa

In [ ]:
## team is abbreviation, all_stunts is all_stunts.csv
def when_they_blitz(team,all_stunts):
    tm = all_stunts.loc[all_stunts.defensiveTeam == team]
    tm.loc[(tm.down == 1)&(tm.yardsToGo==10), 'first10'] = 1
    tm.loc[(tm.down == 2)&(tm.yardsToGo<5), 'secondShort'] = 1
    tm.loc[(tm.down == 2)&(tm.yardsToGo>=5), 'secondLong'] = 1
    tm.loc[(tm.down == 3)&(tm.yardsToGo<5), 'thirdShort'] = 1
    tm.loc[(tm.down == 3)&(tm.yardsToGo>=5), 'thirdLong'] = 1
    tm.loc[(tm.down == 4)&(tm.yardsToGo<5), 'fourthShort'] = 1
    tm.loc[(tm.down == 4)&(tm.yardsToGo>=5), 'fourthLong'] = 1

    
    total_plays = tm.shape[0]
    
    ## stunt rates
    sr_first10 = tm[tm.dummyStunt == 1].first10.sum()/total_plays
    sr_secondShort = tm[tm.dummyStunt == 1].secondShort.sum()/total_plays
    sr_secondLong = tm[tm.dummyStunt == 1].secondLong.sum()/total_plays
    sr_thirdShort = tm[tm.dummyStunt == 1].thirdShort.sum()/total_plays
    sr_thirdLong = tm[tm.dummyStunt == 1].thirdLong.sum()/total_plays
    sr_fourthShort = tm[tm.dummyStunt == 1].fourthShort.sum()/total_plays
    sr_fourthLong = tm[tm.dummyStunt == 1].fourthLong.sum()/total_plays
    
    sr_G2G = tm[(tm.dummyStunt == 1) & (tm.yardsToEZ <= 10)].dummyStunt.count()/total_plays
    sr_redzone = tm[(tm.dummyStunt == 1) & (tm.yardsToEZ <= 20)].dummyStunt.count()/total_plays
    sr_fgRange = tm[(tm.dummyStunt == 1) & (tm.yardsToEZ <= 45)].dummyStunt.count()/total_plays
    
    sr_singleScoreW = tm[(tm.dummyStunt == 1) &
                         (tm.pointDifferential <= 8) &
                         (tm.pointDifferential > 0)].dummyStunt.count()/total_plays
    sr_singleScoreL = tm[(tm.dummyStunt == 1) &
                         (tm.pointDifferential >= -8) &
                         (tm.pointDifferential < 0)].dummyStunt.count()/total_plays
    sr_singleScoreT = tm[(tm.dummyStunt == 1) &
                         (tm.pointDifferential == 0)].dummyStunt.count()/total_plays
    sr_multScoreW = tm[(tm.dummyStunt == 1) & (tm.pointDifferential > 8)].dummyStunt.count()/total_plays
    sr_multScoreL = tm[(tm.dummyStunt == 1) & (tm.pointDifferential < -8)].dummyStunt.count()/total_plays
                       
    sr_quarterOne = tm[(tm.dummyStunt == 1) & (tm.quarter == 1)].dummyStunt.count()/total_plays
    sr_quarterTwo = tm[(tm.dummyStunt == 1) & (tm.quarter == 2)].dummyStunt.count()/total_plays
    sr_quarterThree = tm[(tm.dummyStunt == 1) & (tm.quarter == 3)].dummyStunt.count()/total_plays
    sr_quarterFour = tm[(tm.dummyStunt == 1) & (tm.quarter == 4)].dummyStunt.count()/total_plays
    stunt_rates = [sr_first10,sr_secondShort,sr_secondLong,sr_thirdShort,sr_thirdLong,
            sr_fourthShort,sr_fourthLong,sr_G2G,sr_redzone,sr_fgRange,sr_singleScoreW,
            sr_singleScoreL,sr_singleScoreT,sr_multScoreW,sr_multScoreL,sr_quarterOne,
            sr_quarterTwo,sr_quarterThree,sr_quarterFour]
    
    ## blitz rates
    br_first10 = tm[tm.dummyBlitz == 1].first10.sum()/total_plays
    br_secondShort = tm[tm.dummyBlitz == 1].secondShort.sum()/total_plays
    br_secondLong = tm[tm.dummyBlitz == 1].secondLong.sum()/total_plays
    br_thirdShort = tm[tm.dummyBlitz == 1].thirdShort.sum()/total_plays
    br_thirdLong = tm[tm.dummyBlitz == 1].thirdLong.sum()/total_plays
    br_fourthShort = tm[tm.dummyBlitz == 1].fourthShort.sum()/total_plays
    br_fourthLong = tm[tm.dummyBlitz == 1].fourthLong.sum()/total_plays
    
    br_G2G = tm[(tm.dummyBlitz == 1) & (tm.yardsToEZ <= 10)].dummyBlitz.count()/total_plays
    br_redzone = tm[(tm.dummyBlitz == 1) & (tm.yardsToEZ <= 20)].dummyBlitz.count()/total_plays
    br_fgRange = tm[(tm.dummyBlitz == 1) & (tm.yardsToEZ <= 45)].dummyBlitz.count()/total_plays
    
    br_singleScoreW = tm[(tm.dummyBlitz == 1) &
                         (tm.pointDifferential <= 8) &
                         (tm.pointDifferential > 0)].dummyBlitz.count()/total_plays
    br_singleScoreL = tm[(tm.dummyBlitz == 1) &
                         (tm.pointDifferential >= -8) &
                         (tm.pointDifferential < 0)].dummyBlitz.count()/total_plays
    br_singleScoreT = tm[(tm.dummyBlitz == 1) &
                         (tm.pointDifferential == 0)].dummyBlitz.count()/total_plays
    br_multScoreW = tm[(tm.dummyBlitz == 1) & (tm.pointDifferential > 8)].dummyBlitz.count()/total_plays
    br_multScoreL = tm[(tm.dummyBlitz == 1) & (tm.pointDifferential < -8)].dummyBlitz.count()/total_plays
                       
    br_quarterOne = tm[(tm.dummyBlitz == 1) & (tm.quarter == 1)].dummyBlitz.count()/total_plays
    br_quarterTwo = tm[(tm.dummyBlitz == 1) & (tm.quarter == 2)].dummyBlitz.count()/total_plays
    br_quarterThree = tm[(tm.dummyBlitz == 1) & (tm.quarter == 3)].dummyBlitz.count()/total_plays
    br_quarterFour = tm[(tm.dummyBlitz == 1) & (tm.quarter == 4)].dummyBlitz.count()/total_plays
    
    blitz_rates = [br_first10,br_secondShort,br_secondLong,br_thirdShort,br_thirdLong,
            br_fourthShort,br_fourthLong,br_G2G,br_redzone,br_fgRange,br_singleScoreW,
            br_singleScoreL,br_singleScoreT,br_multScoreW,br_multScoreL,br_quarterOne,
            br_quarterTwo,br_quarterThree,br_quarterFour]
    
    print("Stunt Rates: {}".format(stunt_rates))
    print('-----')
    print("Blitz Rates: {}".format(blitz_rates))
    
    plt.bar([1,2,3,4],stunt_rates[-4:])
    plt.xlabel("Quarter")
    plt.ylabel("Stunt Rate")
    ax = plt.gca()
    ax.set_xticks(np.arange(1,5,1))
    plt.show()
    
    plt.bar([1,2,3,4],blitz_rates[-4:])
    plt.xlabel("Quarter")
    plt.ylabel("Blitz Rate")
    ax = plt.gca()
    ax.set_xticks(np.arange(1,5,1))
    plt.show()
    
    return [stunt_rates,blitz_rates]